<a href="https://colab.research.google.com/github/Xenofon-Chiotopoulos/QOSF_exercises/blob/main/QOSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install qiskit

In [4]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit import Aer, execute
from math import pi
import numpy as np

Excercise 1 quantum multiplyer

In [53]:
def to_bitstring(int1, int2):
  '''Turns ints to bitstring'''
  num1 = str(format(int1, "b"))
  num2 = str(format(int2, "b"))
  len1 = len(num1)
  len2 = len(num2)

  if len1>len2:
    num2= '0'*(len1-len2) + num2
  else:
    num1= '0'*(len2-len1) + num1

  return num1, num2

def input_state(qc, register, n):
  '''performs QFT on register'''
  qc.h(register[n])
  for i in range(0, n):
      qc.cp(pi / float(2**(i + 1)), register[n - (i + 1)], register[n])


def evolve_QFT(qc, register_a, register_b, n, factor):
  '''Evolves |(ψ(a))> to |(ψ(a+b))>'''
  l = len(register_b)
  for i in range(0, n + 1):
      if (n - i) > l - 1:
          pass
      else:
          qc.cp(factor*pi / float(2**(i)), register_b[n - i], register_a[n])

def inv_QFT(qc, register, n):
  '''performs inverse QFT to get output state'''
  for i in range(0, n):
      qc.cp(-1 * pi / float(2**(n - i)), register[i], register[n])
  qc.h(register[n])

def add(register_a, register_b, circ, factor):
  '''Adds qunatum registers by doing a = a+b '''
  n = len(register_a) - 1

  for i in range(0, n + 1):
      input_state(circ, register_a, n - i)
  for i in range(0, n + 1):
      evolve_QFT(circ, register_a, register_b, n - i, factor)
  for i in range(0, n + 1):
      inv_QFT(circ, register_a, i)

IndentationError: ignored

In [50]:
def multiply_numbers(num1, num2):
  sign = 0
  if num1*num2 < 0:
    sign = -1
  elif num1*num2 > 0:
    sign = 1

  num1, num2 = to_bitstring(int1, int2)
  len1, len2 = len(num1), len(num2)
  if len2 > len1:
    num1, num2 = num2, num1
    len2, len1 = len1, len2

  multiplicand = QuantumRegister(len1)
  multiplier = QuantumRegister(len2)
  final_sum = QuantumRegister(len1 + len2)

  cl = ClassicalRegister(len1 + len2)
  d = QuantumRegister(1)
  circ = QuantumCircuit(final_sum, multiplier, multiplicand, d, cl, name="qc")
  circ.x(d)

  for i in range(len1):
    if num1[i] == '1':
        circ.x(multiplicand[len1 - i - 1])

  for i in range(len2):
      if num2[i] == '1':
          circ.x(multiplier[len1 - i - 1])

  multiplier_str = '1'
  while(int(multiplier_str) != 0):
    add(final_sum, multiplicand, circ, 1)
    add(multiplier, d, circ, -1)
    for i in range(len(multiplier)):
        circ.measure(multiplier[i], cl[i])
    result = execute(circ, backend=Aer.get_backend('qasm_simulator'),
                    shots=2).result().get_counts(circ.name)
    multiplier_str = list(result.keys())[0]

  circ.measure(final_sum, cl)
  result = execute(circ, backend=Aer.get_backend('qasm_simulator'),
              shots=2).result().get_counts(circ.name)
  result = list(result.keys())[0]
  #print(circ)
  return int(str(result), 2) * sign

In [39]:
int1 = 1; int2 = 1
res = multiply_numbers(int1,int2)
print(res)

                        ┌───┐                 ┌───┐               ┌─┐   
q518_0: ──────■─────────┤ H ├───────────■─────┤ H ├─■─────────────┤M├───
        ┌───┐ │P(π/2)   └───┘           │     └───┘ │P(-π/2) ┌───┐└╥┘┌─┐
q518_1: ┤ H ├─■────────■────────────────┼───────────■────────┤ H ├─╫─┤M├
        ├───┤  ┌───┐   │                │     ┌───┐   ┌─┐    └───┘ ║ └╥┘
  q517: ┤ X ├──┤ H ├───┼────────■───────┼─────┤ H ├───┤M├──────────╫──╫─
        ├───┤  └───┘   │P(π/2)  │       │P(π) └───┘   └╥┘          ║  ║ 
  q516: ┤ X ├──────────■────────┼───────■──────────────╫───────────╫──╫─
        ├───┤                   │P(-π)                 ║           ║  ║ 
  q519: ┤ X ├───────────────────■──────────────────────╫───────────╫──╫─
        └───┘                                          ║           ║  ║ 
 c19_0: ═══════════════════════════════════════════════╩═══════════╩══╬═
                                                                      ║ 
 c19_1: ═══════════════════════════════════════════

In [33]:
int1 = -4; int2 = 4
res = multiply_numbers(int1,int2)
print(res)

-16


In [34]:
int1 = -4; int2 = -4
res = multiply_numbers(int1,int2)
print(res)

16


In [35]:
int1 = 10; int2 = 10
res = multiply_numbers(int1,int2)
print(res)

100


In [51]:
int1 = 0; int2 = 10
res = multiply_numbers(int1,int2)
print(res)

0
